In [0]:
# For the project we’ll be using OfficeDataProject.csv
# Read data from the file in the DF and perform following analytics on it.
# Print the total number of employees in the company
# Print the total number of departments in the company
# Print the department names of the company
# Print the total number of employees in each department
# Print the total number of employees in each state
# Print the total number of employees in each state in each department
# Print the minimum and maximum salaries in each department and sort salaries in ascending order
# Print the names of employees working in NY state under Finance department whose bonuses are greater than the average bonuses of employees in NY state
# Raise the salaries $500 of all employees whose age is greater than 45
# Create DF of all those employees whose age is greater than 45 and save them in a file

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Mini Project').getOrCreate()
df = spark.read.options(header=True,inferSchema=True).csv('/FileStore/tables/OfficeData.csv')
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [0]:
# 3
df_emp_count = df.select(df.employee_name).count()
df_emp_count

Out[8]: 9

In [0]:
# 4
df_dept_count = df.groupby('department').count().show()
# or
df.select("department").dropDuplicates(["department"]).count()


+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+

Out[22]: 3

In [0]:
# 5
df.select('department').distinct().show()
# or
df.select("department").dropDuplicates(['department']).show()

+----------+
|department|
+----------+
|     Sales|
|   Finance|
| Marketing|
+----------+

+----------+
|department|
+----------+
|     Sales|
|   Finance|
| Marketing|
+----------+



In [0]:
# 6
from pyspark.sql.functions import count
df.groupby('department').agg(count('employee_name').alias('Total Emp')).show()
# or
df1 = df.groupby('department').count().show()

+----------+---------+
|department|Total Emp|
+----------+---------+
|     Sales|        3|
|   Finance|        4|
| Marketing|        2|
+----------+---------+

+----------+-----+
|department|count|
+----------+-----+
|     Sales|    3|
|   Finance|    4|
| Marketing|    2|
+----------+-----+



In [0]:
# 7
df.groupby('state').agg(count('employee_name').alias('total emp')).show()
# or
df.groupby('state').count().show()

+-----+---------+
|state|total emp|
+-----+---------+
|   CA|        4|
|   NY|        5|
+-----+---------+

+-----+-----+
|state|count|
+-----+-----+
|   CA|    4|
|   NY|    5|
+-----+-----+



In [0]:
# 8
df.groupby('state','department').count().show()

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|    1|
|   CA| Marketing|    1|
|   NY|     Sales|    2|
|   CA|   Finance|    2|
|   NY|   Finance|    2|
|   NY| Marketing|    1|
+-----+----------+-----+



In [0]:
# 9
from pyspark.sql.functions import min,max,col
df.groupby('department').agg(min('salary').alias('Min salary'),max('salary').alias('Max salary')).orderBy(col('Min salary').asc(),col('Max salary').asc()).show()

+----------+----------+----------+
|department|Min salary|Max salary|
+----------+----------+----------+
|   Finance|     79000|     99000|
| Marketing|     80000|     91000|
|     Sales|     81000|     90000|
+----------+----------+----------+



In [0]:
# 10
from pyspark.sql.functions import min,max,col,avg

avg_bonus = df.filter(df.state=='NY').groupby('state').agg(avg('bonus').alias('Avg_bonus')).collect()[0]['Avg_bonus']
print(avg_bonus)
df.filter((df.state == 'NY') & (df.department == 'Finance') & (df.bonus > avg_bonus)).show()

17000.0
+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        Scott|   Finance|   NY| 83000| 36|19000|
+-------------+----------+-----+------+---+-----+



In [0]:
# 11 Raise the salaries $500 of all employees whose age is greater than 45
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def raise_salary(age, salary):
    
    if age > 45:
        return salary+500
    return salary

sal_increment = udf(lambda x,y : raise_salary(x,y), IntegerType())

df.show()
df.withColumn('salary',sal_increment(col('age'),col('salary'))).show()


+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86500| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|

In [0]:
# Create DF of all those employees whose age is greater than 45 and save them in a file
df.filter(df.age > 45).write.csv('/FileStore/tables/df_output/age45.csv')